In [1]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

Possibly helpful: [this link](https://github.com/drabastomek/GTC/blob/master/SJ_2020/workshop/1_Setup/Setup.ipynb), and this sample dockerfile from Jordan:

```
FROM mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04

# Install Horovod, temporarily using CUDA stubsddd 
RUN ldconfig /usr/local/cuda/lib64/stubs && \     
# Install AzureML SDK     
pip install --no-cache-dir azureml-defaults && \     
# Install PyTorch     
pip install --no-cache-dir tensorflow==2.0.0b1 tensorflow-gpu==2.0.0b1 keras==2.0.8 matplotlib==3.0.3 seaborn==0.9.0 requests==2.21.0 bs4==0.0.1 imageio==2.5.0 sklearn pandas==0.24.2 numpy==1.16.2 hickle==3.4.3 && \     
# Install Horovod     
pip install --no-cache-dir horovod==0.13.5 && \     ldconfig
```

In [2]:
import shutil

shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [3]:
#import sys
#sys.path.insert(0,'./')
# Install pycocotools
# !git clone https://github.com/cocodataset/cocoapi.git
# !ls
# %cd cocoapi/PythonAPI
# !ls
# !pip install cython
# !pip install numpy
# !python setup.py build_ext install

In [4]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.0.65 (/home/gopalv/anaconda3/envs/azureml/lib/python3.6/site-packages), Requirement.parse('azureml-telemetry==1.0.69.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.0.65.1 (/home/gopalv/anaconda3/envs/azureml/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.69.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.0.65.1 (/home/gopalv/anaconda3/envs/azureml/lib/python3.6/site-packages), Requirement.parse('azureml-core==1.0.69.*')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azurem

SDK version: 1.0.65


In [5]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [6]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-02-26T13:45:16.509000+00:00', 'errors': None, 'creationTime': '2020-02-14T22:38:17.474022+00:00', 'modifiedTime': '2020-02-14T22:39:03.552424+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [8]:
import urllib.request
 
url = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
 
urllib.request.urlretrieve(url, './test.zip')

('./test.zip', <http.client.HTTPMessage at 0x7f64ff6c0668>)

In [9]:
from zipfile import ZipFile

zip = ZipFile(file='./test.zip')
zip.extractall()

In [10]:
!ls PennFudanPed/

Annotation  PNGImages  PedMasks  added-object-list.txt	readme.txt


In [11]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d


In [12]:
ds.upload('./PennFudanPed', target_path='data', overwrite=False)

Uploading an estimated of 512 files
Target already exists. Skipping upload for data/added-object-list.txt
Target already exists. Skipping upload for data/readme.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00001.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00002.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00003.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00004.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00005.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00006.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00007.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00008.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00009.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00010.txt
Target already exists. Skipping upload for data/Annotation/FudanP

Target already exists. Skipping upload for data/Annotation/PennPed00043.txt
Target already exists. Skipping upload for data/Annotation/PennPed00044.txt
Target already exists. Skipping upload for data/Annotation/PennPed00045.txt
Target already exists. Skipping upload for data/Annotation/PennPed00046.txt
Target already exists. Skipping upload for data/Annotation/PennPed00047.txt
Target already exists. Skipping upload for data/Annotation/PennPed00048.txt
Target already exists. Skipping upload for data/Annotation/PennPed00049.txt
Target already exists. Skipping upload for data/Annotation/PennPed00050.txt
Target already exists. Skipping upload for data/Annotation/PennPed00051.txt
Target already exists. Skipping upload for data/Annotation/PennPed00052.txt
Target already exists. Skipping upload for data/Annotation/PennPed00053.txt
Target already exists. Skipping upload for data/Annotation/PennPed00054.txt
Target already exists. Skipping upload for data/Annotation/PennPed00055.txt
Target alrea

Target already exists. Skipping upload for data/PNGImages/FudanPed00062.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00063.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00064.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00065.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00066.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00067.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00068.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00069.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00070.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00071.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00072.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00073.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00074.png
Target alrea

Target already exists. Skipping upload for data/PedMasks/FudanPed00010_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00011_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00012_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00013_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00014_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00015_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00016_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00017_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00018_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00019_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00020_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00021_mask.png
Target already exists. Skipping upload f

Target already exists. Skipping upload for data/PedMasks/PennPed00041_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00042_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00043_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00044_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00045_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00046_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00047_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00048_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00049_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00050_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00051_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00052_mask.png
Target already exists. Skipping upload for data/PedM

$AZUREML_DATAREFERENCE_2ec3d98c00e14f84acdd9245384b370d

In [13]:
from azureml.core import Dataset
# create a FileDataset pointing to files in 'animals' folder and its subfolders recursively
datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

In [14]:
!git clone https://github.com/pytorch/vision.git

!git checkout v0.3.0

%cd vision
!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../
%cd ..

fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git.
/home/gopalv/pytorch-object/vision
/home/gopalv/pytorch-object


In [15]:
import shutil

os.listdir()

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./vision/references/detection/'+ file + '.py', project_folder)
shutil.copy('./conda-dependencies.yml', project_folder)


'./pytorch-peds/conda-dependencies.yml'

In [16]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

In [34]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

conda_dep = CondaDependencies()

env = Environment(name = 'maskr-pytorch')
# env.docker.base_image = 'Custom-PyTorch'


# conda_dep.add_channel("conda_forge")
conda_dep.add_channel("pytorch")

conda_packages = ['torchvision=0.5.0', 'pytorch=1.4', 'pycocotools=2.0.0']

for package in conda_packages:
    conda_dep.add_conda_package(package)
#add_pip_package("pycocotools")
env.python.conda_dependencies=conda_dep

env.name = 'pycoco-env'
env.register(workspace=ws)
# my_env = Environment(name='maskr-docker')
# my_env.docker.enabled = True
# with open("dockerfiles/Dockerfile1", "r") as f:
#     dockerfile_contents_of_your_base_image=f.read()
# my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 
# my_env.docker.base_image = None # 'mcr.microsoft.com/azureml/base-gpu:openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
# my_env.python.interpreter_path = '/opt/miniconda/bin/python'
# my_env.python.user_managed_dependencies = True

# from azureml.core import Environment

# my_env = Environment.from_conda_specification(name = "myenv",
#                                              file_path = "conda-dependencies.yml")

{
    "name": "pycoco-env",
    "version": "6",
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "python": {
        "userManagedDependencies": false,
        "interpreterPath": "python",
        "condaDependenciesFile": null,
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "conda-forge",
                "pytorch"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
                        "azureml-defaults"
                    ]
                },
                "torchvision=0.5.0",
                "pytorch=1.4",
                "pycocotools=2.0.0"
            ],
            "name": "azureml_bbdc3d39486675fbc1afacaad8324ddc"
        }
    },
    "docker": {
        "enabled": false,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseDockerfile": null,
        "sharedVolumes"

In [35]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

# follow pattern from here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-environments-for-training

# Add training script to run config
runconfig = ScriptRunConfig(source_directory=project_folder, script="script.py")

# Attach compute target to run config
runconfig.run_config.target = cluster_name
#runconfig.run_config.target = "local"

# Attach environment to run config
runconfig.run_config.environment = env

# Submit run 
run = experiment.submit(runconfig)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1582752117_52d02a8b,
Type: azureml.scriptrun,
Status: Starting)


In [36]:
# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1582752117_52d02a8b', 'target': 'gpu-cluster1', 'status': 'Starting', 'warnings': [{'message': "This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.\nPlease enable Docker in the environment section of your run configuration to stop seeing this warning message."}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '4ae476f4-9231-4ec4-a685-993b17408ff8', 'azureml.git.repository_uri': 'git@github.com:gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'git@github.com:gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': '89073737c0ef57eb3e3872056f3567ff31b69d91', 'mlflow.source.git.commit': '89073737c0ef57eb3e3872056f3567ff31b69d91', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': [], 'sourceDirectoryDataStore': None, 'fra

In [37]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582752117_52d02a8b
Web View: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1582752117_52d02a8b

Streaming azureml-logs/20_image_build_log.txt

2020/02/26 21:22:17 Downloading source code...
2020/02/26 21:22:18 Finished downloading source code
2020/02/26 21:22:19 Creating Docker network: acb_default_network, driver: 'bridge'

2020/02/26 21:22:28 Successfully set up Docker network: acb_default_network
2020/02/26 21:22:28 Setting up Docker configuration...
2020/02/26 21:22:32 Successfully set up Docker configuration
2020/02/26 21:22:32 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/26 21:22:33 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/26 21:22:33 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/26 21:22:


olefile-0.46         | 31 KB     |            |   0% 
olefile-0.46         | 31 KB     | ########## | 100% 

pytorch-1.4.0        | 432.9 MB  |            |   0% 
pytorch-1.4.0        | 432.9 MB  |            |   0% 
pytorch-1.4.0        | 432.9 MB  |            |   0% 
pytorch-1.4.0        | 432.9 MB  |            |   0% 
pytorch-1.4.0        | 432.9 MB  |            |   0% 
pytorch-1.4.0        | 432.9 MB  |            |   0% 
pytorch-1.4.0        | 432.9 MB  | 1          |   1% 
pytorch-1.4.0        | 432.9 MB  | 1          |   2% 
pytorch-1.4.0        | 432.9 MB  | 2          |   3% 
pytorch-1.4.0        | 432.9 MB  | 3          |   3% 
pytorch-1.4.0        | 432.9 MB  | 4          |   4% 
pytorch-1.4.0        | 432.9 MB  | 5          |   5% 
pytorch-1.4.0        | 432.9 MB  | 6          |   6% 
pytorch-1.4.0        | 432.9 MB  | 6          |   7% 
pytorch-1.4.0        | 432.9 MB  | 7          |   7% 
pytorch-1.4.0        | 432.9 MB  | 8          |   8% 
pytorch-1.4.0        | 432


pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  78% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.9 MB  | #######8   |  79% 
pytorch-1.4.0        | 432.


pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########6  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  87% 
pytorch-1.4.0        | 432.9 MB  | ########7  |  88% 
pytorch-1.4.0        | 432.

...


libxcb-1.13          | 396 KB    |            |   0% 
libxcb-1.13          | 396 KB    | ########9  |  90% 
libxcb-1.13          | 396 KB    | ########## | 100% 

glib-2.53.5          | 8.2 MB    |            |   0% 
glib-2.53.5          | 8.2 MB    |            |   0% 
glib-2.53.5          | 8.2 MB    | 8          |   9% 
glib-2.53.5          | 8.2 MB    | ###4       |  35% 
glib-2.53.5          | 8.2 MB    | #####8     |  58% 
glib-2.53.5          | 8.2 MB    | #######5   |  75% 
glib-2.53.5          | 8.2 MB    | ########7  |  88% 
glib-2.53.5          | 8.2 MB    | #########6 |  97% 
glib-2.53.5          | 8.2 MB    | ########## | 100% 

pyparsing-2.4.6      | 59 KB     |            |   0% 
pyparsing-2.4.6      | 59 KB     | ########## | 100% 

fontconfig-2.12.1    | 429 KB    |            |   0% 
fontconfig-2.12.1    | 429 KB    | ########## | 100% 

zstd-1.4.4           | 989 KB    |            |   0% 
zstd-1.4.4           | 989 KB    | ########4  |  84% 
zstd-1.4.4           | 

  Attempting uninstall: numpy
    Found existing installation: numpy 1.11.3
    Uninstalling numpy-1.11.3:
      Successfully uninstalled numpy-1.11.3

#
# To activate this environment, use:
# > source activate /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc
#
# To deactivate an active environment, use:
# > source deactivate
#


Removing intermediate container 2753cf303b31
 ---> 856dfa9fe856
Step 9/15 : ENV PATH /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/bin:$PATH
 ---> Running in c6536cf07a89
Removing intermediate container c6536cf07a89
 ---> f9efeae66aae
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc
 ---> Running in 0dfd324609b5
Removing intermediate container 0dfd324609b5
 ---> 2060a0525857
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib:$LD_LIBRARY_PATH
 ---> Running in 0b37b599a8d6
Removing intermediate container 0b37b599a8d6
 ---> 1c4247e2f091
Step 12/15 : COPY 

94daaa4fbb84841e8046396b1196a000a80d3b7747c0b53aa7e05f25be4abbec
2020/02/26 21:46:25 Version: 3.0.01140.0002 Branch: merge Commit: 93399ed1
2020/02/26 21:46:26 /dev/infiniband/uverbs0 found (implying presence of InfiniBand)?: false
2020/02/26 21:46:26 sshd runtime has already been installed in the container
ssh-keygen: /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/libcrypto.so.1.0.0: no version information available (required by ssh-keygen)
ssh-keygen: /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/libcrypto.so.1.0.0: no version information available (required by ssh-keygen)
bash: /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/libtinfo.so.5: no version information available (required by bash)
bash: /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/libtinfo.so.5: no version information available (required by bash)
2020-02-26T21:46:39Z Job environment preparation succeeded on 10.0.0.4. Output: 
>>>   2020/02/26 21:44:55 Version: 3.0.01140.000


Streaming azureml-logs/75_job_post-tvmps_11d3ec2ac49c4be15ee8d56d19b414614ed975b7876a96cae1e1181a93d9603e_d.txt

bash: /azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/libtinfo.so.5: no version information available (required by bash)
Starting job release. Current time:2020-02-26T21:46:56.729672
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 203
Job release is complete. Current time:2020-02-26T21:46:59.057300

Execution Summary
RunId: pytorch-peds_1582752117_52d02a8b
Web View: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/aifxdemo/providers/Microsoft.MachineLearningServices/workspaces/gopalv-ws/experiments/pytorch-peds/runs/pytorch-peds_1582752117_52d02a8b

Warnings:
This compute target type doesn't support non-Docker runs; overriding run configuration to enable Docker.
Please enable Docker in the environment section of your run 

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ImportError: libGL.so.1: cannot open shared object file: No such file or directory",
        "details": [],
        "debugInfo": {
            "type": "ImportError",
            "message": "libGL.so.1: cannot open shared object file: No such file or directory",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/gopalv-ws/azureml/pytorch-peds_1582752117_52d02a8b/mounts/workspaceblobstore/azureml/pytorch-peds_1582752117_52d02a8b/azureml-setup/context_manager_injector.py\", line 127, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/python3.6/runpy.py\", line 263, in run_path\n    pkg_name=pkg_name, script_name=fname)\n  File \"/azureml-envs/azureml_bbdc3d39486675fbc1afacaad8324ddc/lib/python3.